<a href="https://colab.research.google.com/github/ShikharV010/gist_daily_runs/blob/main/Wednesday_supplyOPS_Mailer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from sqlalchemy import create_engine

In [ ]:
# --- PostgreSQL connection info ---
pg_params = {
    'host': 'gw-postgres-dev.celzx4qnlkfp.us-east-1.rds.amazonaws.com',
    'database': 'gw_prod',
    'user': 'airbyte_user',
    'password': 'airbyte_user_password',
    'port': '5432'
}

In [ ]:
# SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{pg_params['user']}:{pg_params['password']}@{pg_params['host']}:{pg_params['port']}/{pg_params['database']}"
)

In [ ]:
# --- SQL logic: matches your Metabase question 4855 ---
query = """
WITH cte_base_data AS (
  SELECT
    "gist"."writerallocation_contractedapproved"."Campaign ID" AS campaign_id,
    "gist"."writerallocation_contractedapproved"."URL" AS url,
    CAST("gist"."writerallocation_contractedapproved"."Next Billing Date" AS DATE) AS next_billing_date,
    "gist"."writerallocation_contractedapproved"."Topics Approved" AS topics_approved,
    CAST("gist"."writerallocation_contractedapproved"."Outstanding on day" AS  int) AS outstanding_on_day,
    "gist"."writerallocation_contractedapproved"."Status" AS "Status",
    "gist"."writerallocation_contractedapproved"."Type" AS "Type",
    "Gist Billablecampaignscsm - Campaign"."cp_id" AS cp_id,
    "Gist Billablecampaignscsm - Campaign"."name" AS campaign__name,
    "Gist Billablecampaignscsm - Campaign"."url" AS campaign__url,
    "Gist Billablecampaignscsm - Campaign"."csm" AS campaign__csm,
    "Gist Billablecampaignscsm - Campaign"."clean_url" AS campaign__clean_url,
    "Gist Billablecampaignscsm - Campaign"."onboarding_date" AS campaign__onboarding_date,
    "Gist Billablecampaignscsm - Campaign"."days_since_onboarding" AS campaign__days_since_onboarding,
    "Gist Billablecampaignscsm - Campaign"."workflow" AS campaign__workflow
  FROM
    "gist"."writerallocation_contractedapproved"
  LEFT JOIN "gist"."gist_billablecampaignscsm" AS "Gist Billablecampaignscsm - Campaign"
    ON "gist"."writerallocation_contractedapproved"."Campaign ID" = "Gist Billablecampaignscsm - Campaign"."cp_id"
  WHERE
    "gist"."writerallocation_contractedapproved"."Status" = 'Active'
    AND "gist"."writerallocation_contractedapproved"."Type" = 'Product'
),

cte_flags AS (
  SELECT *,
      -- Low Topics Flag
    CASE
      WHEN topics_approved <= 8
           AND outstanding_on_day > topics_approved
      THEN 'Low topics'
      ELSE NULL
    END AS low_topics_flag,
    CASE
      WHEN outstanding_on_day > topics_approved
           AND next_billing_date IS NOT NULL
           AND (next_billing_date - CURRENT_DATE) <= 10
      THEN 'Contract end date approaching'
      ELSE NULL
    END AS imbalanced_topics_extended_flag
  FROM cte_base_data
)

SELECT *, CONCAT_WS(', ',low_topics_flag,imbalanced_topics_extended_flag)  AS reasons
FROM cte_flags
WHERE low_topics_flag = 'Low topics'	OR imbalanced_topics_extended_flag = 'Contract end date approaching'
ORDER BY 1;
"""


In [ ]:
# --- Read into DataFrame ---
df = pd.read_sql_query(query, engine)
print("Data fetched:", df.shape)

Data fetched: (14, 18)


In [ ]:
# --- Email configuration ---
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
EMAIL_SENDER = 'Gist.support@gushwork.ai'  # Replace with your sender email
EMAIL_PASSWORD = 'fbpu luxs wxcb ypul'   # Use Gmail App Password if using Gmail

In [ ]:
# --- Send email function ---
def send_email(to_email, subject, html_body):
    msg = MIMEText(html_body, 'html')
    msg['Subject'] = subject
    msg['From'] = EMAIL_SENDER
    msg['To'] = to_email

    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_SENDER, EMAIL_PASSWORD)
        server.sendmail(EMAIL_SENDER, [to_email], msg.as_string())

In [ ]:
print(df)

                             campaign_id                             url  \
0   013300d9-d7e1-4cf7-8b88-e16f02d5c600                      topmate.io   
1   136d8360-69ee-43fa-ac78-90afbfdbfa84                       compup.io   
2   1dccf5ae-2f10-4e47-b07e-df0b8d34ca39                     vedantu.com   
3   531f6adf-eae2-4768-90a7-0f65d3bc9537                khandelwalaw.com   
4   5e42fd93-7338-4f21-8a0b-0da9175f9b6e                     planacan.io   
5   61f708d1-7421-4ffb-a3be-0967aa4b072c  https://www.expandastands.com/   
6   638a483b-3076-45e3-bc1b-b0d122cfdc0a                   akirolabs.com   
7   6e7d3978-3663-42cc-9f1d-805be86905d8                getclockwise.com   
8   cfdf0373-877d-4105-ad82-102fa76abf02               procoffeegear.com   
9   d23bc80c-3e09-40bc-bc50-5855c6dd43a0              www.cloudtech.com/   
10  d53170e8-2568-4072-bfac-bc8a37229f3d                 returnprime.com   
11  d616fe7e-669a-4a2f-bf54-828d6c7a6c07                        insia.ai   
12  dfd5a6fc

In [ ]:
# --- Group and email per CSM ---
for csm, group in df.groupby("campaign__csm"):
    if pd.isna(csm):
        continue

    csm_first_name = csm.split('@')[0].split('.')[0].title()

    low_topics_rows = group[group['reasons'].str.contains('Low topics', na=False)]
    imbalanced_rows = group[group['reasons'].str.contains('Contract end date approaching', na=False)]

    html_body = f"<p>Hi {csm_first_name},</p>"

    if not low_topics_rows.empty:
        html_body += """
<p>As of today, the following accounts have fewer than 8 approved topics.
Please ensure that the required number of topics are approved no later than <b>Friday afternoon</b>.
Failure to do so will result in <b>no writers</b> being assigned to these accounts for the coming writer allocation.</p>
<table border="1" cellpadding="5">
    <tr>
        <th>Client</th>
        <th>Topics Approved</th>
        <th>Outstanding Blogs</th>
    </tr>
"""
        for _, row in low_topics_rows.iterrows():
            html_body += f"""
    <tr>
        <td>{row['campaign__clean_url']}</td>
        <td>{row['topics_approved']}</td>
        <td>{row['outstanding_on_day']}</td>
    </tr>
"""
        html_body += "</table><br>"

    if not imbalanced_rows.empty:
        html_body += """
<p>As of today, the following accounts have contract end dates before the end of next week and have fewer approved topics
than the number of blogs currently outstanding. Please ensure that the appropriate number of topics are approved before the
coming Friday afternoon in order to fulfill the contracted blog deliveries before the end of the cycle.</p>
<table border="1" cellpadding="5">
    <tr>
        <th>Client</th>
        <th>Contract End Date</th>
        <th>Days Remaining</th>
        <th>Topics Approved</th>
        <th>Outstanding Blogs</th>
    </tr>
"""
        for _, row in imbalanced_rows.iterrows():
            days_remaining = (pd.to_datetime(row['next_billing_date']) - pd.Timestamp.today().normalize()).days
            html_body += f"""
    <tr>
        <td>{row['campaign__clean_url']}</td>
        <td>{row['next_billing_date']}</td>
        <td>{days_remaining}</td>
        <td>{row['topics_approved']}</td>
        <td>{row['outstanding_on_day']}</td>
    </tr>
"""
        html_body += "</table><br>"

    html_body += "<p>Regards,<br>Gush Information System</p>"

    send_email(csm, "Topics Approved Alert", html_body)

print("✅ All CSM emails sent.")

✅ All CSM emails sent.


In [ ]:
# --- Summary Email to Tushar ---
summary_recipient = "tushar.kumar@gushwork.ai,"
summary_subject = "Summary: Topics Approved Alert"

# Add a CC variable
cc_recipient = "bigrah.prasad@gushwork.ai"

# Filter all rows that need attention
summary_df = df[df['reasons'].notna()]

# Build HTML table rows
summary_rows = ""
for _, row in summary_df.iterrows():
    summary_rows += f"""
    <tr>
        <td style='border: 1px solid #ddd; padding: 8px;'>{row['campaign__clean_url']}</td>
        <td style='border: 1px solid #ddd; padding: 8px;'>{row['campaign__csm']}</td>
        <td style='border: 1px solid #ddd; padding: 8px;'>{row['next_billing_date']}</td>
        <td style='border: 1px solid #ddd; padding: 8px;'>{row['topics_approved']}</td>
        <td style='border: 1px solid #ddd; padding: 8px;'>{row['outstanding_on_day']}</td>
        <td style='border: 1px solid #ddd; padding: 8px;'>{row['reasons']}</td>
    </tr>
    """

# HTML Body
summary_html = f"""
<p>Hi Tushar,</p>
<p>Please find below the accounts that have fewer than 8 topics approved or have fewer topics approved than content left to be done:</p>
<table style='border-collapse: collapse; width: 100%;'>
    <tr style='background-color: #f2f2f2;'>
        <th style='border: 1px solid #ddd; padding: 8px;'>Client</th>
        <th style='border: 1px solid #ddd; padding: 8px;'>CSM</th>
        <th style='border: 1px solid #ddd; padding: 8px;'>Contract End Date</th>
        <th style='border: 1px solid #ddd; padding: 8px;'>Topics Approved</th>
        <th style='border: 1px solid #ddd; padding: 8px;'>Outstanding</th>
        <th style='border: 1px solid #ddd; padding: 8px;'>Issue</th>
    </tr>
    {summary_rows}
</table>
<br><br>
<p>Regards,<br>Gush Information System</p>
"""

# Plain-text Body
summary_text = "Hi Tushar,\n\nPlease find below the accounts that have fewer than 8 topics approved or have fewer topics approved than content left to be done:\n\n"
for _, row in summary_df.iterrows():
    summary_text += f"- {row['campaign__clean_url']}, CSM: {row['campaign__csm']}, Contract End Date: {row['next_billing_date']}, Topics Approved: {row['topics_approved']}, Outstanding: {row['outstanding_on_day']}, Issue: {row['reasons']}\n"

summary_text += "\nRegards,\nGush Information System"

# Send Summary Email
msg = MIMEText(summary_html, "html")  # Change to "plain" if you prefer text version
msg["Subject"] = summary_subject
msg["From"] = EMAIL_SENDER
msg["To"] = summary_recipient
msg["Cc"] = cc_recipient

# build a flat list of all recipients
# if summary_recipient might be comma-separated, split & strip:
to_addrs = [addr.strip()
            for addr in summary_recipient.split(",") if addr.strip()]
to_addrs.append(cc_recipient)

with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
    server.starttls()
    server.login(EMAIL_SENDER, EMAIL_PASSWORD)
    server.sendmail(EMAIL_SENDER, to_addrs, msg.as_string())

print(f"📩 Summary email sent to {summary_recipient} and CC’d to {cc_recipient}.")

📩 Summary email sent to tushar.kumar@gushwork.ai, and CC’d to bigrah@gushwork.ai.
